- Building a CNN from scratch using keras for emotion detection using our live camera feed

# imports

In [5]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import os

In [6]:
# setting up some convenience variables
num_classes = 5
img_rows, img_cols = 48,48
batch_size = 32

In [ ]:
# using the fer2013 dataset from kaggle
train_data_dir = "train"
validation_data_dir = "validation"